## Load Data

In [2]:
from federated_gbdt.models.gbdt.private_gbdt import PrivateGBDT
from experiments.experiment_helpers.data_loader import DataLoader
from sklearn.metrics import roc_auc_score

In [3]:
dataloader = DataLoader()

# Default is 70/30 split
X_train, X_test, y_train, y_test = dataloader.load_datasets(["adult"], return_dict=False)[0]

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')
Counter({0: 24720, 1: 7841})
[Data Loader] Adult Class Balance: 0.2408095574460244 



In [4]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
4740,16,3,6674,9,12,4,9,1,4,1,0,0,39,38
14871,23,5,3441,11,8,2,2,0,4,1,0,0,49,38
4990,16,5,5432,7,11,0,11,1,4,1,0,0,59,38
24778,28,5,20859,9,12,2,9,0,4,1,0,0,39,38
14388,4,3,10480,11,8,4,7,2,4,0,0,0,29,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24901,2,3,5927,2,7,4,7,3,4,1,0,25,19,38
4646,13,3,18632,11,8,2,13,0,4,1,0,0,39,38
8988,15,3,13149,14,14,2,9,0,4,1,0,50,59,38
22187,42,3,4326,15,9,2,3,0,4,1,0,0,11,38


## XGBoost Training (No DP)

In [6]:
xgb_model = PrivateGBDT(num_trees=100, epsilon=0)
xgb_model = xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.9226979537264549

## DP-XGBoost (FEVERLESS)

In [7]:
dp_xgb_model = PrivateGBDT(num_trees=100, epsilon=1, dp_method="gaussian_cdp")
dp_xgb_model = dp_xgb_model.fit(X_train, y_train)
y_pred = dp_xgb_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.8108485034839067

## DP-TR XGBoost

In [9]:
dp_tr_model = PrivateGBDT(num_trees=100, epsilon=1, split_method="totally_random", 
                           sketch_type="uniform", dp_method="gaussian_cdp")

dp_tr_model = dp_tr_model.fit(X_train, y_train)
y_pred = dp_tr_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.8859277936602712